# 42577 Introduction to Business Analytics Project

**Georgios Panagiotopoulos s223306**

## Introduction And Initials Visualizations

The term sustainable mobility goes far beyond reducing emissions. The transport sector has the potential to improve the lives and livelihoods of billions of people. However, as well as meeting people’s needs today, the sector must be ready to respond to future generations’ expectations: this is the essence of sustainable development.

Following this year’s topic, sustainable mobility. We have been assigned with the task of predict the total demand for the bike sharing system “BlueBikes” based in Boston, for 2 hours in the future. 

As context BlieBikes is a company created in July 2011 and located in Metro Boston. Since it creations it have grown exponentially, from 3203 annual members to over 20000 members as of today:
![memberships%20and%20rideships.png](attachment:memberships%20and%20rideships.png)

As always we start by importing all the libraries and functions we are going to use for the data analysis and visualisation as well as for the prediction challenge. *****

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import main as main
import numpy as np
#import holidays
import seaborn as sns
from collections import Counter
import matplotlib.pyplot as plt
from tabulate import tabulate
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
%matplotlib inline

In [3]:
# our data with the target variable
data = pd.read_csv('BikeSharing_Bluebikes2022.csv', index_col=0)

/home/georgep/anaconda3/envs/MachineLearning/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,postal code
0,597,2022-01-01 00:00:25.1660,2022-01-01 00:10:22.1920,178,MIT Pacific St at Purrington St,42.359573,-71.101295,74,Harvard Square at Mass Ave/ Dunster,42.373268,-71.118579,4923,Subscriber,02139
1,411,2022-01-01 00:00:40.4300,2022-01-01 00:07:32.1980,189,Kendall T,42.362428,-71.084955,178,MIT Pacific St at Purrington St,42.359573,-71.101295,3112,Subscriber,02139
2,476,2022-01-01 00:00:54.8180,2022-01-01 00:08:51.6680,94,Main St at Austin St,42.375603,-71.064608,356,Charlestown Navy Yard,42.374125,-71.054812,6901,Customer,02124
3,466,2022-01-01 00:01:01.6080,2022-01-01 00:08:48.2350,94,Main St at Austin St,42.375603,-71.064608,356,Charlestown Navy Yard,42.374125,-71.054812,5214,Customer,02124
4,752,2022-01-01 00:01:06.0520,2022-01-01 00:13:38.2300,19,Park Dr at Buswell St,42.347241,-71.105301,41,Packard's Corner - Commonwealth Ave at Brighto...,42.352261,-71.123831,2214,Subscriber,02215


Shape Of The Dataset

In [5]:
data.shape

(2305735, 14)

In [6]:
#transforming the dates from object to datetime
for date_column in ['starttime','stoptime']:
    data[date_column] = pd.to_datetime(data[date_column], format='%Y-%m-%d %H:%M:%S')

In [7]:
'''
DatetimeInterval function: 
'''
df15 = main.DatetimeInterval(data, freq='15Min')
df30 = main.DatetimeInterval(data, freq='30Min')
df60 = main.DatetimeInterval(data, freq='60Min')
df120 = main.DatetimeInterval(data, freq='120Min')
picks15 = main.Lagging(df15, 15)
picks30 = main.Lagging(df30, 30)
picks60 = main.Lagging(df60, 60)
picks120 = main.Lagging(df120, 120)

In [8]:
df15.head()

,usertype_Customer,usertype_Subscriber,pickups,month,hour,minute,"lag(pickups,60-75)","lag(pickups,75-90)","lag(pickups,90-105)","lag(pickups,105-120)"
starttime,,,,,,,,,,
2022-01-01 00:00:00,7.0,23.0,30.0,1,0,0,NaN,NaN,NaN,NaN
2022-01-01 00:15:00,6.0,30.0,36.0,1,0,15,NaN,NaN,NaN,NaN
2022-01-01 00:30:00,7.0,22.0,29.0,1,0,30,NaN,NaN,NaN,NaN
2022-01-01 00:45:00,8.0,22.0,30.0,1,0,45,NaN,NaN,NaN,NaN
2022-01-01 01:00:00,5.0,16.0,21.0,1,1,0,NaN,NaN,NaN,NaN


In [9]:
picks15.head()

,pickups,month,hour,minute,"lag(pickups,60-75)","lag(pickups,75-90)","lag(pickups,90-105)","lag(pickups,105-120)"
starttime,,,,,,,,
2022-01-01 02:00:00,27.0,1,2,0,30.0,29.0,36.0,30.0
2022-01-01 02:15:00,14.0,1,2,15,21.0,30.0,29.0,36.0
2022-01-01 02:30:00,16.0,1,2,30,22.0,21.0,30.0,29.0
2022-01-01 02:45:00,16.0,1,2,45,24.0,22.0,21.0,30.0
2022-01-01 03:00:00,30.0,1,3,0,25.0,24.0,22.0,21.0


In [ ]:
fig, axs = plt.subplots(len([picks15,picks30,picks60,picks120]), figsize=(16,12))
fig.suptitle('Pickups of bikes distribution during months')

for i, df in enumerate([picks15,picks30,picks60,picks120]):
    axs[i].plot(df['pickups'])

axs[0].set_title('15 minutes')
axs[1].set_title('30 minutes')
axs[2].set_title('60 minutes')
axs[3].set_title('120 minutes')

plt.tight_layout()

In [ ]:
fig, axes = plt.subplots(len([picks15,picks30,picks60,picks120]), figsize=(16,12))
fig.suptitle('Pickups of bikes per hour of day')

for i, df in enumerate([picks15,picks30,picks60,picks120]):
    sns.boxplot(ax=axes[i], x='hour',y='pickups', data=df)

axes[0].set_title('15 minutes')
axes[1].set_title('30 minutes')
axes[2].set_title('60 minutes')
axes[3].set_title('120 minutes')

plt.tight_layout()

In [ ]:
fig, axes = plt.subplots(len([picks15,picks30,picks60,picks120]), figsize=(16,12))
fig.suptitle('Pickups of bikes per month')

for i, df in enumerate([picks15,picks30,picks60,picks120]):
    sns.boxplot(ax=axes[i],x='month',y='pickups', data=df)

axes[0].set_title('15 minutes')
axes[1].set_title('30 minutes')
axes[2].set_title('60 minutes')
axes[3].set_title('120 minutes')

plt.tight_layout()

## Prediction challenge 

In [ ]:
def ModelRunningResults(models,models_name, first_row, picks, weather_columns):
    results = {}
    table = [first_row]
    pipe = main.PredictionPipeline(picks,weather_columns)
    fi_list = []
    X_train, X_test, y_train, y_test = pipe.PredictionDataPreperation()
    for model, model_name in zip(models,models_name):
        CV_score, CV_MSE, test_score, test_MSE, fi = pipe.BackTestingPrediction(model, model_name, X_train, X_test, y_train, y_test)
        results[model_name] = {'Test Score':test_score, 'Test MSE':test_MSE,  'Features Importance':fi}
        row = [model_name, test_score, test_MSE]
        table.append(row)      
        fi_list.append(fi)
    return table, fi_list

In [ ]:
models = [LinearRegression(),Ridge(),Lasso(),RandomForestRegressor(),ExtraTreesRegressor()]
models_name = ['LinearRegression','RidgeRegression','LassoRegression','RandomForestRegressor','ExtraTreesRegressor']
first_row = ['Model for 15','Final Test Score','Final Test RMSE']
weather_columns = []
print('The Prediction based on {} minutes intervals \n'.format(15))
table15, features_importance15 = ModelRunningResults(models,models_name, first_row, picks15, weather_columns)

In [ ]:
print(tabulate(table15, headers='firstrow',colalign=("left",), floatfmt=".4f"))

In [ ]:
print('Featutes importance 15 min')
for model in models_name:
    print(model)
    for i,f in enumerate(features_importance15):
        print(f[i],'\n')

In [ ]:
models = [LinearRegression(),Ridge(),Lasso(),RandomForestRegressor(),ExtraTreesRegressor()]
models_name = ['LinearRegression','RidgeRegression','LassoRegression','RandomForestRegressor','ExtraTreesRegressor']
first_row = ['Model for 30min','Final Test Score','Final Test RMSE']
weather_columns = []
print('The Prediction based on {} minutes intervals \n'.format(30))
table30, features_importance30 = ModelRunningResults(models,models_name, first_row, picks30,weather_columns)

In [ ]:
print(tabulate(table30,headers='firstrow',colalign=("left",), floatfmt=".4f"))

In [ ]:
print('Featutes importance 30 min')
for model in models_name:
    print(model)
    for i,f in enumerate(features_importance30):
        print(f[i],'\n')

In [ ]:
models = [LinearRegression(),Ridge(),Lasso(),RandomForestRegressor(),ExtraTreesRegressor()]
models_name = ['LinearRegression','RidgeRegression','LassoRegression','RandomForestRegressor','ExtraTreesRegressor']
first_row = ['Model for 60','Final Test Score','Final Test RMSE']
weather_columns = []
print('The Prediction based on {} minutes intervals \n'.format(60))
table60, features_importance60  = ModelRunningResults(models,models_name, first_row, picks60,[])

In [ ]:
print(tabulate(table60,headers='firstrow',colalign=("left",), floatfmt=".4f"))

In [ ]:
print('Featutes importance 60 min')
for i,model in enumerate(models_name):
    print(model, features_importance60[i])

In [ ]:
models = [LinearRegression(),Ridge(),Lasso(),RandomForestRegressor(),ExtraTreesRegressor()]
models_name = ['LinearRegression','RidgeRegression','LassoRegression','RandomForestRegressor','ExtraTreesRegressor']
first_row = ['Model for 120','Final Test Score','Final Test RMSE']
weather_columns = []
print('The Prediction based on {} minutes intervals \n'.format(120))
table120, features_importance120 = ModelRunningResults(models,models_name, first_row, picks120,[])

In [ ]:
print(tabulate(table120,headers='firstrow',colalign=("left",), floatfmt=".4f"))

In [ ]:
print('Featutes importance 120 min')
for model in models_name:
    print(model)
    for i,f in enumerate(features_importance120):
        print(f[i],'\n')